In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
import cv2
import matplotlib.pyplot as plt
import os
from sklearn.preprocessing import LabelEncoder 

In [ ]:
import zipfile
zip_ref = zipfile.ZipFile("/content/data_1 (2).zip", 'r')
zip_ref.extractall()
zip_ref.close()

In [ ]:
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
dirs = "data/train/"
img_size = 60

data = []
for name in os.listdir(dirs):
    for f in os.listdir(dirs+name):
        f = cv2.imread(os.path.join(dirs+name, f))
        faces = face_cascade.detectMultiScale(f,1.3,5)
        for x,y,w,h in faces:
            img = f[y:y+h, x:x+w]
            img = cv2.resize(img, (img_size,img_size))
            data.append((img, name))
            
df = pd.DataFrame(data, columns=["image", "name"])
print("Length:",len(df))
#df.head

In [ ]:
dirs = "data/val/"

data = []
for name in os.listdir(dirs):
    for f in os.listdir(dirs+name):
        f = cv2.imread(os.path.join(dirs+name, f))
        faces = face_cascade.detectMultiScale(f,1.3,5)
        for x,y,w,h in faces:
            img = f[y:y+h, x:x+w]
            img = cv2.resize(img, (img_size,img_size))
            data.append((img, name))
            
df_test = pd.DataFrame(data, columns=["image", "name"])
print("Test size: ", len(df_test))

In [ ]:
le = LabelEncoder()
le.fit(df["name"].values)

In [ ]:
x_train = list(df.image.values)
x_train = np.array(x_train)
x_train = x_train/255
print(x_train.shape)

y_train = le.transform(df["name"].values)
print(y_train.shape)

In [ ]:
x_test = list(df_test.image.values)
x_test = np.array(x_test)
x_test = x_test/255
print(x_test.shape)

y_test = le.transform(df_test["name"].values)
print(y_test.shape)

In [ ]:
people_num = len(np.unique(y_train))
people_num

In [ ]:
def triplet_loss(y_true, y_pred, alpha = 0.2):
    total_lenght = y_pred.shape.as_list()[-1]
    anchor, positive, negative = y_pred[:,:int(1/3*total_lenght)], y_pred[:,int(1/3*total_lenght):int(2/3*total_lenght)], y_pred[:,int(2/3*total_lenght):]
    
    pos_dist = tf.reduce_sum(tf.square(anchor - positive), axis=-1)
    neg_dist = tf.reduce_sum(tf.square(anchor - negative), axis=-1)
    basic_loss = pos_dist - neg_dist + alpha
    loss = tf.reduce_sum(tf.maximum(basic_loss,0.0))
    return loss

In [ ]:
def generate_triplets(x, y, num_same = 4, num_diff = 4):
    anchor_images = np.array([]).reshape((-1,)+ x.shape[1:])
    same_images = np.array([]).reshape((-1,)+ x.shape[1:])
    diff_images = np.array([]).reshape((-1,)+ x.shape[1:])
    
    for i in range(len(y)):
        point = y[i]        
        anchor = x[i]
        
        same_pairs = np.where(y == point)[0]
        same_pairs = np.delete(same_pairs , np.where(same_pairs == i))
        diff_pairs = np.where(y != point)[0]
               
        same = x[np.random.choice(same_pairs,num_same)]
        diff = x[np.random.choice(diff_pairs,num_diff)]
        
        anchor_images = np.concatenate((anchor_images, np.tile(anchor, (num_same * num_diff, 1, 1, 1) )), axis = 0)
                                       
        for s in same:
            same_images = np.concatenate((same_images, np.tile(s, (num_same, 1, 1, 1) )), axis = 0)
            
        diff_images = np.concatenate((diff_images, np.tile(diff, (num_diff, 1, 1, 1) )), axis = 0)
        
    return anchor_images, same_images, diff_images

In [ ]:
anchor_images, same_images, diff_images = generate_triplets(x_train,y_train, num_same= 10, num_diff=10)
print(anchor_images.shape, same_images.shape, diff_images.shape)

In [ ]:
idx = 90
plt.subplot(1,3,1)
plt.imshow(anchor_images[idx])

plt.subplot(1,3,2)
plt.imshow(same_images[idx])

plt.subplot(1,3,3)
plt.imshow(diff_images[idx])

In [ ]:
def get_model():
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Conv2D(64, kernel_size=3, strides=2, padding='same', input_shape=(img_size,img_size,3), activation='relu'))
    model.add(tf.keras.layers.Conv2D(128, kernel_size=3, strides=2, padding='same', activation='relu'))
    model.add(tf.keras.layers.Conv2D(64, kernel_size=3, strides=2, padding='same', activation='relu'))
    model.add(tf.keras.layers.Conv2D(64, kernel_size=1, strides=2, padding='same', activation='relu'))
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(512, activation='relu'))
    model.add(tf.keras.layers.Dropout(0.1))
    model.add(tf.keras.layers.Dense(256, activation='relu'))
    model.add(tf.keras.layers.Dropout(0.2))
    model.add(tf.keras.layers.Dense(128))
              
    model.summary()
    return model

In [ ]:

anchor_input = tf.keras.layers.Input((img_size, img_size, 3), name='anchor_input')
positive_input = tf.keras.layers.Input((img_size, img_size, 3), name='positive_input')
negative_input = tf.keras.layers.Input((img_size, img_size, 3), name='negative_input')

shared_dnn = get_model()

encoded_anchor = shared_dnn(anchor_input)
encoded_positive = shared_dnn(positive_input)
encoded_negative = shared_dnn(negative_input)

merged_vector = tf.keras.layers.concatenate([encoded_anchor, encoded_positive, encoded_negative],
                                            axis=-1, name='merged_layer')

model = tf.keras.Model(inputs=[anchor_input,positive_input, negative_input], outputs=merged_vector)
model.summary()
model.compile(loss=triplet_loss, optimizer="adam")

In [ ]:
weight_dir = "weight_tripletloss_model"
if not os.path.exists(weight_dir):
    os.mkdir(weight_dir)
    
checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath=weight_dir+'/checkpoint-{epoch:02d}.hdf5')

In [ ]:
Y_dummy = np.empty((anchor_images.shape[0],1))

model.fit([anchor_images,same_images,diff_images],y=Y_dummy, batch_size=16, epochs=50, callbacks=[checkpoint])

In [ ]:
anchor_model = tf.keras.Model(inputs = anchor_input, outputs=encoded_anchor)

In [ ]:
pred = anchor_model.predict(x_train)
pred.shape

In [ ]:
def img_encoding(imgpath,model):
  f = cv2.imread(os.path.join(imgpath, f))
  faces = face_cascade.detectMultiScale(f,1.3,5)
  for x,y,w,h in faces:
    img = f[y:y+h, x:x+w]
    img = cv2.resize(img, (img_size,img_size))
  pred=model.predict(img)
  return pred

In [ ]:
'''database={}
database["Howard"]=img_encoding("/content/Howard.jpg",anchor_model)
database["Leonard"]=img_encoding("/content/Leonard.jpg",anchor_model)
database["Penny"]=img_encoding("/content/Penny.jpg",anchor_model)
database["Sheldon"]=img_encoding("/content/sheldon.jpg",anchor_model)'''



In [ ]:
def encode_image(model ,img):
    encode = model.predict(img.reshape((1,)+ img.shape))
    encode=encode/255
    print(encode)
    return encode

In [ ]:
def encode(model,dir):
  test_image1 = cv2.imread(dir)
  print(test_image1.shape)
  test_image_gray1 = cv2.cvtColor(test_image1, cv2.COLOR_BGR2GRAY)
  faces = face_cascade.detectMultiScale(test_image_gray1,1.3,5)
  for x,y,w,h in faces:
    img = test_image1[y:y+h, x:x+w]
    img = cv2.resize(img, (60,60))
  return encode_image(model,img)
database={}

database["jerryseinfield"]=encode(anchor_model,"/content/jerryseinfield.jpg")
database["Madonna"]=encode(anchor_model,"/content/Madonna.jpg")
database["benafflek"]=encode(anchor_model,"/content/benafflek.jpg")

In [ ]:
name_dict = {}
for i in set(df["name"].values):
    z = df[df["name"] == i].image
    img = np.array(list(z))/255
    enc = np.zeros((1,128))
    for j in range(len(z)):
        enc += encode_image(anchor_model,img[j])

    enc = enc/len(z)
    name_dict[i] = enc
#print(name_dict)

In [ ]:
def who_is_it(image_path, database, model):
    
    encoding = encode_image(model,image_path)
    print(encoding)
    min_dist = 10000
    for (name, db_enc) in database.items():
        
        dist = np.linalg.norm(encoding-db_enc)
        if dist<min_dist:
            min_dist = dist
            identity = name
    
    if min_dist >0.1:
        print("Not in the database.")
    else:
        print ("it's " + str(identity) + ", the distance is " + str(min_dist))
        
    return min_dist, identity

In [ ]:
test_image1 = cv2.imread("/content/MV5BMTI4MzIxMTk0Nl5BMl5BanBnXkFtZTcwOTU5NjA0Mg@@._V1_UY1200_CR85,0,630,1200_AL_.jpg")
print(test_image1.shape)
test_image_gray1 = cv2.cvtColor(test_image1, cv2.COLOR_BGR2GRAY)
faces = face_cascade.detectMultiScale(test_image_gray1,1.3,5)
for x,y,w,h in faces:
  img = test_image1[y:y+h, x:x+w]
  img = cv2.resize(img, (60,60))
  plt.imshow(img)
  print(who_is_it(img,database,anchor_model))
